Vediamo le soluzioni di alcuni esercizi richiesti dagli studenti.

### **Esercizio 2.1.**
Scrivere una funzione `genera_lista` che prende un intero positivo `n` e restituisce una lista contenente la lista `[1; 2; ... n]`. Nel caso in cui `n` sia minore o uguale di zero restituisce la lista vuota.

In [2]:
(*Per generare una lista in maniera ricorsiva, bisogna individuare quale sia il sottoproblema su cui
  chiamarsi ricorsivamente: in questo caso, per generare una lista [1; .. n] è necessario prima
  generare una lista [1; .. n-1]*)
let rec genera_lista n =
    if n <= 0
        then [] (*caso base*)
    else
        genera_lista(n-1)@[n] (*caso induttivo*)


(*La funzione precedente fa uso dell'operatore @, che ha complessità lineare, 
  quindi la funzione così definita ha complessità quadratica. Usando l'operatore
  ::, che ha complessità costante, si ottiene però una lista "al rovescio", dall'elemento
  n fino all'elemento 1.*)
let rec genera_lista2 n =
    if n <= 0 then [] 
    else n::genera_lista2(n-1)
  
(*Per ottenere una lista orientata correttamente, ma mantenedo la complessità lineare di genera_lista2,
  è necessario usare una funzione ausiliaria, il cui codice è esattamente quello di genera_lista2.
  genera_lista3 non dovrà far altro che invertire il risultato della funzione aux.*)
let genera_lista3 n = 
    let rec aux n = 
        if n <= 0 then []
        else n::aux (n-1)
    in
    List.rev (aux n);;
        
genera_lista 10;;
genera_lista2 10;;
genera_lista3 10;;

val genera_lista : int -> int list = <fun>


val genera_lista2 : int -> int list = <fun>


val genera_lista3 : int -> int list = <fun>


- : int list = [1; 2; 3; 4; 5; 6; 7; 8; 9; 10]


- : int list = [10; 9; 8; 7; 6; 5; 4; 3; 2; 1]


- : int list = [1; 2; 3; 4; 5; 6; 7; 8; 9; 10]


### **Esercizio 2.6.**
Scrivere una funzione `ord` che prende una lista e verifica (restituendo `true` o `false`) se i suoi elementi sono ordinati in modo crescente.

In [1]:
(*Per scrivere la funzione ord in maniera ricorsiva, bisogna effettuare pattern matching 
  sui primi **due** elementi della lista.*)
let rec ord list = match list with
    |[] -> true
    |[x] -> true
    |x::(y::tail as rest) -> (x <= y) && ord (rest);;

val ord : 'a list -> bool = <fun>


Per scrivere la funzione sfruttando le funzioni di ordine superiore, possiamo usare la `fold_left` con una apposita funzione `f`. Va ricordato che `f` prende in input due argomenti, l'accumulatore e l'attuale elemento della lista, e restituisce in output un nuovo accumulatore. Come spesso accade, l'accumulatore è il risultato parziale che stiamo calcolando: la funzione `f`, chiamata sull'elemento n-esimo, riceve il risultato calcolato sui primi n-1 elementi, esegue delle operazioni in base all'elemento n-esimo, e restituisce il risultato calcolato sui primi n elementi.
 
In questo caso, il risultato parziale altro non è che un valore booleano che vale inizialmente `true`, e rimane `true` solo se l'elemento n-esimo è maggiore dell'elemento (n-1)-esimo, altrimenti diventa `false`, e rimarrà `false` per tutte le chiamate di f successive.

Quando `f` deve calcolare il nuovo risultato sull'elemento n-esimo, però, necessita anche dell'informazione sull'esemento (n-1)-esimo, quello visitato precedentemente. L'unico modo per conservare questa informazione fra una chiamata e l'altra di `f`, è conservarlo all'interno dell'acccumulatore. L'accumulatore dovrà quindi contenere due informazioni, da aggiornare con ogni chiamata di `f`:
- Il risultato parziale fin'ora calcolato
- L'elemento visitato precedentemente, da confrontare con quello corrente

Il valore iniziale dell'accumulatore deve essere anche esso una coppia di un booleano e un elemento della lista. Poichè la prima chiamata di `f` confronterà `curr` (il primo elemento)  con `prev` (il valore iniziale), come valore iniziale possiamo scegliere il primo elemento stesso, in maniera tale che `curr>=prev` restituisca sicuramente `true`.

Vale la pena notare che le variabili che cambiano fra le diverse chiamate di `f`, cioè `curr`, `prev` e `result`, sono esattamente le variabili che cambierebbero scrivendo un ciclo for in un qualsiasi linguaggio imperativo: `f` è il corpo del for, che viene eseguito uguale a ogni iterazione, gli argomenti di `f` sono lo stato che muta fra una iterazione e l'altra, e `fold_left` corrisponde al costrutto for in se, che itera più volte lo stesso codice.

Il risultato di questa operazione di fold_l sarà una coppia, (così come il valore iniziale dell'accumulatore e il  valore di ritorno di `f`). Dei due valori contenuti nel risultato della fold_r, solo il primo è utile, il secondo può essere scartato. 

In [ ]:
let ord2 list = 
    let f (result, prev) curr = 
        if (curr >= prev) && result
            then (true, curr) 
            else (false, curr)
    in
    let res, _ = List.fold_left f (true, List.hd list) list in
    res;;

Se volessimo scrivere la funzione `ord` utilizzando `fold_right` e non `fold_left`, l'algoritmo è molto simile, ma è necessario utilizzare l'ultimo elemento della lista (e non il primo) come accumulatore iniziale.

In [2]:
let ord3 list = 
    let last_element = List.hd (List.rev list) in
     let res, _ = List.fold_right (
        fun curr (output, prev) -> if (curr <= prev) && output
                                        then (true, curr) 
                                        else (false, curr)
    ) list (true, last_element) in
    res;;


let l = [1;3;5;2;5;];;
ord l;;
ord2 l;;
ord3 l;;

val ord2 : 'a list -> bool = <fun>


val ord3 : 'a list -> bool = <fun>


val l : int list = [1; 3; 5; 2; 5]


- : bool = false


- : bool = false


- : bool = false
